In [2]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from scipy.stats.stats import pearsonr 
from operator import itemgetter
import math as mh

# Support Functions

In [3]:
#we should divide the total summ value with 2(33(max value of date_block_num)//12(months))
def getBaseSaleValueForMonth(item_id, month):
    if isInTraining(item_id):
        base_month_sum = (sum(df[(df.item_id == item_id) & ((df.date_block_num%12) == month)].item_cnt_day))/2
        return base_month_sum
    else:
        category_id = getCategoryIdForItem(item_id)
        base_category_month_sum = (sum(df[(df.item_category_id == category_id) & 
                                          ((df.date_block_num%12) == month)].item_cnt_day))/2
        return base_category_month_sum

def isInTraining(item_id):
    if item_id in items_training:
        return True
    return False

def getCategoryIdForItem(item_id):
    return category_data_dict[item_id]

def getNormalizedAvgSalesTwoMonth(item_id, month):
    total_month_sum = float(getBaseSaleValueForMonth(item_id, month) + getBaseSaleValueForMonth(item_id, month+1))/2
    total_sum = -1
    if isInTraining(item_id):
        total_sum = sum(df[(df.item_id == item_id)].item_cnt_day)
    else:
        total_sum = sum(df[(df.item_id == item_id)].item_cnt_day)
    if total_sum > 0:
        return total_month_sum/total_sum
    return 0
    
def getPercentageChangeInTwoMonths(item_id, month):
    month1_sales = getBaseSaleValueForMonth(item_id, month)
    month2_sales = getBaseSaleValueForMonth(item_id, month+1)
    if month1_sales > 0:
        return ((month1_sales-month2_sales)/(month1_sales))*100
    return 0


In [8]:
raw_train_data = pd.read_csv("sales_train.csv")
category_data = pd.read_csv("items.csv").loc[:,["item_id", "item_category_id"]]
raw_test_data = pd.read_csv("test.csv")
df = raw_train_data.join(category_data.set_index("item_id"), on = "item_id").drop("date", axis = 1)
# category_data = category_data.sample(frac=0.1, replace=False)
df = df.sample(frac=0.01, replace=False)
# raw_test_data = raw_test_data.sample(frac=0.1, replace=False)
category_data_dict = {x[0]:x[1] for x in category_data.itertuples(index = False)}
# df = df.sample(frac=0.5, replace = True)

In [10]:
# create a common set of item_ids
items_training = set()
items_testing = set()
for index, item in raw_train_data.iterrows():
    items_training.add(item["item_id"])
print(len(items_training))
for index, item in raw_test_data.iterrows():
    items_testing.add(item["item_id"])
print(len(items_testing))
items = list(set(items_testing.union(items_training)))

21807


KeyboardInterrupt: 

# Creating new feature DataFrame

In [ ]:
items_df = pd.DataFrame(columns=["item_id","base_sale_nov","base_sale_oct","norm_avg_sale_oct_nov",
                                "norm_avg_sale_sept_oct","norm_avg_sale_aug_sep",
                                "percent_change_oct_nov","percent_change_sept_oct","percent_change_aug_sept",
                                 "percent_change_jul_aug"])

In [ ]:
#Nov date_block_num = 10
for index,item_id in enumerate(items_training):
    items_df.loc[index] = [int(item_id), 
                          getBaseSaleValueForMonth(item_id, 10),
                          getBaseSaleValueForMonth(item_id, 9),
                          getNormalizedAvgSalesTwoMonth(item_id, 9),
                          getNormalizedAvgSalesTwoMonth(item_id, 8),
                          getNormalizedAvgSalesTwoMonth(item_id, 7),
                          getPercentageChangeInTwoMonths(item_id, 9),
                          getPercentageChangeInTwoMonths(item_id, 8),
                          getPercentageChangeInTwoMonths(item_id, 7),
                          getPercentageChangeInTwoMonths(item_id, 6)]

items_df.head()

# prep test data

In [ ]:
test_items_df = pd.DataFrame(columns=["item_id","base_sale_nov","base_sale_oct","norm_avg_sale_oct_nov",
                                "norm_avg_sale_sept_oct","norm_avg_sale_aug_sep",
                                "percent_change_oct_nov","percent_change_sept_oct","percent_change_aug_sept",
                                 "percent_change_jul_aug"])

In [243]:
copy_df = df.copy(deep = True)
#join is behaving weird
final_data = items_df.join(df.set_index("item_id"), on = "item_id", how = "inner")
# final_data = df.append(items_df)

In [244]:
copy_df.drop("item_id", axis = 1, inplace = True)
final_data.drop("item_id", axis = 1, inplace = True)

In [245]:
for column in ["shop_id", "item_category_id"]:
    final_data = pd.concat([final_data ,pd.get_dummies(final_data[column], prefix = column)], axis = 1)
    final_data.drop([column], axis = 1, inplace = True)
    copy_df = pd.concat([copy_df ,pd.get_dummies(copy_df[column], prefix = column)], axis = 1)
    copy_df.drop([column], axis = 1, inplace = True)
# final_data = final_data[not (mh.isnan(final_data.base_sale_nov))]
df_train, df_test = train_test_split(final_data, test_size = 0.2)
copy_df_train, copy_df_test = train_test_split(copy_df, test_size = 0.2)

In [246]:
# final_data.columns.values
# x = final_data.base_sale_oct

In [258]:
lm = linear_model.LinearRegression()
# lm = RandomForestClassifier()
df_train_Y = df_train.filter(regex="item_cnt_day")
df_train_X = df_train.drop("item_cnt_day", axis=1, inplace = False)
df_test_Y = df_test.filter(regex="item_cnt_day")
df_test_X = df_test.drop("item_cnt_day", axis=1, inplace = False)


In [259]:
# clf.fit(df_train_X, df_train_Y)
lm.fit(df_train_X, df_train_Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [260]:
# df_test_pred = clf.predict(df_test_X)
df_test_pred = lm.predict(df_test_X)

In [261]:
mean_absolute_error(df_test_Y.values, df_test_pred)

0.4497559461999006

In [262]:
mean_squared_error(df_test_Y.values, df_test_pred)

4.084318600012128

# Predictions sans new features

In [263]:
lm2 = linear_model.LinearRegression()
# lm2 = RandomForestClassifier()
copy_df_train_Y = copy_df_train.filter(regex="item_cnt_day")
copy_df_train_X = copy_df_train.drop("item_cnt_day", axis=1, inplace = False)
copy_df_test_Y = copy_df_test.filter(regex="item_cnt_day")
copy_df_test_X = copy_df_test.drop("item_cnt_day", axis=1, inplace = False)

In [264]:
lm2.fit(copy_df_train_X, copy_df_train_Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [265]:
copy_df_test_pred = lm2.predict(copy_df_test_X)

In [266]:
mean_absolute_error(copy_df_test_Y.values, copy_df_test_pred)

0.403088733560303

In [267]:
mean_squared_error(copy_df_test_Y.values, copy_df_test_pred)

3.611833768828795

In [257]:
# copy_df.columns.values

# Testing set predictions

In [296]:
test_items_df = pd.DataFrame(columns=["item_id","base_sale_nov","base_sale_oct","norm_avg_sale_oct_nov",
                                "norm_avg_sale_sept_oct","norm_avg_sale_aug_sep",
                                "percent_change_oct_nov","percent_change_sept_oct","percent_change_aug_sept",
                                 "percent_change_jul_aug"])

In [297]:
for index,item_id in enumerate(items_testing):
    if mh.isnan(getBaseSaleValueForMonth(item_id, 10)):
        continue
    test_items_df.loc[index] = [int(item_id), 
                          getBaseSaleValueForMonth(item_id, 10),
                          getBaseSaleValueForMonth(item_id, 9),
                          getNormalizedAvgSalesTwoMonth(item_id, 9),
                          getNormalizedAvgSalesTwoMonth(item_id, 8),
                          getNormalizedAvgSalesTwoMonth(item_id, 7),
                          getPercentageChangeInTwoMonths(item_id, 9),
                          getPercentageChangeInTwoMonths(item_id, 8),
                          getPercentageChangeInTwoMonths(item_id, 7),
                          getPercentageChangeInTwoMonths(item_id, 6)]
test_items_df.head()

,item_id,base_sale_nov,base_sale_oct,norm_avg_sale_oct_nov,norm_avg_sale_sept_oct,norm_avg_sale_aug_sep,percent_change_oct_nov,percent_change_sept_oct,percent_change_aug_sept,percent_change_jul_aug
0,8192.0,0.0,0.5,0.020833,0.020833,0.020833,100.0,0.0,100.0,0.0
1,8193.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,100.0
2,16388.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,100.0
3,16395.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,16396.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [298]:
testing_final_data = items_df.join(raw_test.set_index("item_id"), on = "item_id", how = "inner")

NameError: name 'raw_test' is not defined

In [ ]:
testing_final_data.drop("item_id", axis = 1, inplace = True)

In [ ]:
for column in ["shop_id", "item_category_id"]:
    testing_final_data = pd.concat([testing_final_data ,pd.get_dummies(testing_final_data[column], prefix = column)], axis = 1)
    testing_final_data.drop([column], axis = 1, inplace = True)

In [ ]:
testing_final_data.head

# OLD CODE

In [3]:
df = df[(df.item_cnt_day <=100) & (df.item_cnt_day>0)]
df_train, df_test = train_test_split(df, test_size=0.2)
df_train = df_train.sample(frac=0.5, replace = True)

In [4]:
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,0,59,22154,999.00,1.0,37
1,0,25,2552,899.00,1.0,58
3,0,25,2554,1709.05,1.0,58
4,0,25,2555,1099.00,1.0,56
5,0,25,2564,349.00,1.0,59


In [5]:
cat_sum = {}
for category in pd.unique(df.item_category_id):
    category_date_list = []
    for date_val in pd.unique(df.date_block_num):
        sum_value = sum(df[(df.item_category_id == category) & (df.date_block_num == date_val)].item_cnt_day) 
        category_date_list.append([date_val, sum_value])
    cat_sum[category] = category_date_list

In [23]:
for cat in cat_sum.items():
    date_b = [x[0] for x in cat[1]]
    cnt_b = [x[1] for x in cat[1]]
    ax = plt.subplot()
    plt.scatter([x%12 for x in date_b], cnt_b, s=1)
    plt.title(cat[0])
    plt.show()

dict_values([[[0, 6132.0], [1, 6340.0], [2, 6113.0], [3, 6199.0], [4, 5823.0], [5, 6783.0], [6, 8644.0], [7, 8742.0], [8, 7703.0], [9, 6989.0], [10, 7441.0], [11, 9989.0], [12, 7545.0], [13, 6044.0], [14, 6334.0], [15, 7721.0], [16, 7199.0], [17, 6212.0], [18, 5669.0], [19, 6017.0], [20, 4992.0], [21, 5096.0], [22, 5200.0], [23, 7598.0], [24, 5985.0], [25, 4863.0], [26, 5117.0], [27, 4677.0], [28, 3300.0], [29, 3761.0], [30, 3299.0], [31, 3436.0], [32, 2999.0], [33, 4069.0]], [[0, 292.0], [1, 334.0], [2, 378.0], [3, 318.0], [4, 347.0], [5, 273.0], [6, 357.0], [7, 286.0], [8, 320.0], [9, 374.0], [10, 448.0], [11, 709.0], [12, 453.0], [13, 383.0], [14, 330.0], [15, 381.0], [16, 303.0], [17, 367.0], [18, 370.0], [19, 375.0], [20, 438.0], [21, 406.0], [22, 587.0], [23, 812.0], [24, 537.0], [25, 480.0], [26, 506.0], [27, 397.0], [28, 302.0], [29, 369.0], [30, 378.0], [31, 286.0], [32, 383.0], [33, 452.0]], [[0, 268.0], [1, 299.0], [2, 297.0], [3, 334.0], [4, 255.0], [5, 328.0], [6, 320.0], 

In [7]:
corelation_list = []
for cat0 in cat_sum.items():
    for cat1 in cat_sum.items():
        if cat0[0] == cat1[0]:
            continue
        vec0 = [x[1] for x in cat0[1]]
        vec1 = [x[1] for x in cat1[1]]
        res = pearsonr(vec0, vec1)
        corelation_list.append([res[0],cat0[0],cat1[0]])
# print(corelation_list)
# List of pearson corelation between categories in terms of monthly trend
sorted(corelation_list, key = itemgetter(0), reverse = True)

[[0.985000476802942, 21, 4],
 [0.985000476802942, 4, 21],
 [0.9787858792691323, 57, 43],
 [0.9787858792691323, 43, 57],
 [0.9738877528635039, 59, 82],
 [0.9738877528635039, 82, 59],
 [0.9738448341965901, 59, 77],
 [0.9738448341965901, 77, 59],
 [0.9726663773547924, 4, 66],
 [0.9726663773547924, 66, 4],
 [0.971260282546896, 82, 81],
 [0.971260282546896, 81, 82],
 [0.9711551647007576, 40, 43],
 [0.9711551647007576, 43, 40],
 [0.9685959430795983, 59, 66],
 [0.9685959430795983, 66, 59],
 [0.9683466267288556, 77, 66],
 [0.9683466267288556, 66, 77],
 [0.9651967189324978, 21, 66],
 [0.9651967189324978, 66, 21],
 [0.9616895311708894, 77, 82],
 [0.9616895311708894, 82, 77],
 [0.9598028572364862, 59, 4],
 [0.9598028572364862, 4, 59],
 [0.9589305048786283, 79, 34],
 [0.9589305048786283, 34, 79],
 [0.9571065524171805, 15, 6],
 [0.9571065524171805, 6, 15],
 [0.9556439280858982, 62, 2],
 [0.9556439280858982, 2, 62],
 [0.9543718342686571, 77, 81],
 [0.9543718342686571, 81, 77],
 [0.9534116008393206, 

In [8]:
len(corelation_list)

6972

In [12]:
cluster_list = []
cluster_threshold = 0.7
for tup in corelation_list:
#     print(cluster_list)
    if tup[0] >= cluster_threshold:
        #check if tup[1] or tup[2] exists in list of cluster_set
        if not cluster_list :
#             print("1")
#             print(tup)
            cat_cluster = set()
            cat_cluster.add(int(tup[1]))                  
            cat_cluster.add(int(tup[2]))
            cluster_list.append(cat_cluster)
        else :
            in_cat = False
            for cat_set in cluster_list:
                if ((tup[1] in cat_set) and (tup[2] not in cat_set)):
                    cat_set.add(int(tup[2]))
                    in_cat = True
#                     print("2")
#                     print(tup)
                elif ((tup[2] in cat_set) and (tup[1] not in cat_set)):
                    cat_set.add(int(tup[1]))
                    in_cat = True
#                     print("3")
#                     print(tup)
            if not in_cat:
                cat_cluster = set()
                cat_cluster.add(int(tup[1]))
                cat_cluster.add(int(tup[2]))
                cluster_list.append(cat_cluster)
#                 print("4")
#                 print(cluster_list)
                    
                

In [13]:
len(cluster_list)

52

In [19]:
final_set = []
i = 0
while i < len(cluster_list):
        if len(cluster_list[0].intersection(cluster_list[1])) > 0 :
            final_set.append(cluster_list[0].union(cluster_list[1]))
            i+= 2
        else:
            i+=1
for x,item in enumerate(final_set):
    print(x,item)

0 {0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 27, 30, 31, 34, 35, 37, 38, 40, 41, 43, 45, 49, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 79, 81, 82}
1 {0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 27, 30, 31, 34, 35, 37, 38, 40, 41, 43, 45, 49, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 79, 81, 82}
2 {0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 27, 30, 31, 34, 35, 37, 38, 40, 41, 43, 45, 49, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 79, 81, 82}
3 {0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 27, 30, 31, 34, 35, 37, 38, 40, 41, 43, 45, 49, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 79, 81, 82}
4 {0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25